In [24]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.max_columns = 15

guest_id = int(input('insert a user id : '))

insert a user id : 20


In [25]:
data = pd.read_table('data/u.data', sep='\t', usecols=['user id', 'movie id', 'rating'])
data

,user id,movie id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [26]:
data = data.pivot_table(index='movie id', columns='user id', values='rating', fill_value=0)
data

user id,1,2,3,4,5,6,7,...,937,938,939,940,941,942,943
movie id,,,,,,,,,,,,,,,
1,5,4,0,0,4,4,0,...,0,4,0,0,5,0,0
2,3,0,0,0,3,0,0,...,0,0,0,0,0,0,5
3,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0
4,3,0,0,0,0,0,5,...,0,0,0,2,0,0,0
5,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
1680,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0


當余弦值越大則余弦夾角越小，表示兩向量越相似；反之，余弦值越小則余弦夾角越大，表示兩向量差異越大。
余弦值 = 1 時，余弦夾角為 0 度。
余弦值 = 0 時，余弦夾角為 90 度。
余弦值 = -1 時，兩向量為相反方向。

In [ ]:
sim = DataFrame()
for i in data.columns:
    sim[i] = DataFrame(cosine_similarity([data[guest_id]], [data[i]]))

sim

,1,2,3,4,5,6,7,...,937,938,939,940,941,942,943
0,0.241498,0.060499,0.050696,0.086565,0.240738,0.181222,0.215534,...,0.028904,0.26077,0.206737,0.202662,0.121379,0.180197,0.216799


In [ ]:
plt.scatter(sim, sim, s=5, alpha=0.5)
plt.show()

In [ ]:
top10_near = sim.T.sort_values(by=0, ascending=False).head(11).T
top10_near

In [ ]:
drop_df = data[data[guest_id] > 0][guest_id].to_frame('rating').T
drop_df

In [ ]:
mult_rec = DataFrame()
for i in top10_near.columns:
    if i != guest_id:
        mult_rec[i] = data[i] * (1 + top10_near[i][0])

data = mult_rec.drop(drop_df)
data

In [ ]:
data = data.T.sum().sort_values(ascending=False).to_frame('recommend rate').head(10)
data

In [ ]:
item = pd.read_table('data/u.item', sep='|', usecols=['movie id', 'movie title', 'release date'])
item

In [ ]:
item = item.set_index(['movie id'])
item

In [ ]:
pd.concat([data, item], axis=1, join_axes=[data.index])